# This file is to convert dicom file to nii. This dicom which is chosen is the nearest time to the date of patient.##

In [1]:
import shutil
import os
import dicom2nifti
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
os.environ['CONDA_DEFAULT_ENV']

'neuro'

In [3]:
ls

README.md                          extractVolumesForSubtypes.ipynb
aimedSubFolderNameListMake.sh      list.json
all.json                           makeSpecificPeopleCSV.ipynb
atrophyModelBuild.ipynb            merge_folder.ipynb
convertFreesurferOutput2CSV.ipynb  nohup.out
csvFiles/                          updateTags.sh
dicomToNii.ipynb                   uploadNiiToBrainIo.ipynb
downloadFreesurferOutput.sh        uploadNiiToBrainIo.sh


In [4]:
# dicom2nifti.convert_directory?

In [5]:
# dicom2nifti.convert_directory('/home/ubuntu/Desktop/ADNI_MRI_Image/small_MM1/128_S_2130/Accelerated_Sag_IR-FSPGR/2018-09-14_14_02_40.0/S727985','/home/ubuntu/Desktop/ADNI_MRI_Image/nii_small_MM1/nii_128_S_2130')

In [8]:
csvPath = "/home/ubuntu/Desktop/ADNI_MRI_Image/neuroScript/csvFiles"

## This part is for MCI data

In [5]:
MCI_amyloid_pos = pd.read_csv(os.path.join(csvPath,'MCI_amyloid_pos.csv'))
MCI_all = pd.read_csv(os.path.join(csvPath,'MCI_all.csv'))

In [65]:
MCI_amyloid_pos

,PTID,EXAMDATE,ABETA_bl,AV45
0,128_S_2003,2010-07-23,NaN,1.16466
1,031_S_2022,2010-08-05,929.2,1.31359
2,072_S_2037,2010-08-24,NaN,1.43899
3,099_S_2042,2010-08-25,827.6,1.06558
4,002_S_2043,2010-09-07,NaN,1.20402
...,...,...,...,...
302,099_S_6632,2019-01-17,NaN,1.18938
303,168_S_6634,2019-02-21,NaN,1.60110
304,023_S_6702,2019-04-26,NaN,1.20806
305,051_S_6719,2019-05-15,NaN,1.23803


In [7]:
MCI_all

,PTID,EXAMDATE
0,128_S_2002,2010-06-16
1,128_S_2003,2010-07-23
2,031_S_2022,2010-08-05
3,072_S_2037,2010-08-24
4,099_S_2042,2010-08-25
...,...,...
623,137_S_6693,2019-12-16
624,052_S_6832,2020-01-09
625,052_S_6844,2020-01-13
626,024_S_6846,2020-01-23


In [10]:
pathRoot = "/home/ubuntu/Desktop/ADNI_MRI_Image/MM1"
niiAmyloidPosPathRoot = '/home/ubuntu/Desktop/ADNI_MRI_Image/nii_AmyloidPos_MCI'
niiAllPathRoot = '/home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI'

In [9]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-SPGR' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    CM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(CM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-FSPGR' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

128_S_2003


NameError: name 'niiPathRoot' is not defined

In [11]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiAllPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-SPGR' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    CM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(CM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiAllPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in MCI_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-FSPGR' in serType:
                    MCI_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,MCI_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiAllPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

128_S_2002
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/128_S_2002/SAG_MPRAGE_NO_ANGLE/2013-06-26_19_00_00.0/S193698 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/128_S_2002 2010-06-16
128_S_2003
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/128_S_2003/MPRAGE/2010-07-01_19_11_50.0/S88768 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/128_S_2003 2010-07-23
031_S_2022
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/031_S_2022/MPRAGE_SENSE2/2010-07-14_15_27_25.0/S89099 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/031_S_2022 2010-08-05
072_S_2037
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/072_S_2037/MPRAGE/2010-08-09_11_31_56.0/S90266 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/072_S_2037 2010-08-24
099_S_2042
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/099_S_2042/MPRAGE/2010-08-18_11_48_20.0/S90767 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/099_S_2042 2010-08-25
002_S_2043
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_2043/MPRAGE_SENSE2/2010-08-31_08_41_49.0/S91526 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/002_S_2043

013_S_2324
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/013_S_2324/MPRAGE_SENSE2/2011-03-25_13_38_07.0/S102469 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/013_S_2324 2011-04-04
141_S_2333
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/141_S_2333/MPRAGE/2011-03-24_15_25_18.0/S102362 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/141_S_2333 2011-04-01
027_S_2336
129_S_2347
126_S_2360
130_S_2373
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/130_S_2373/MPRAGE_SENSE2/2011-05-02_08_52_04.0/S107322 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/130_S_2373 2011-05-11
029_S_2376
036_S_2378
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/036_S_2378/MPRAGE/2011-05-26_14_15_31.0/S110057 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/036_S_2378 2011-07-06
022_S_2379
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/022_S_2379/MPRAGE/2011-05-13_09_12_51.0/S109000 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/022_S_2379 2011-06-03
009_S_2381
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/009_S_2381/MPRAGE/2011-05-18_10_42_41.0/S109343 /home/ubuntu/Deskt

153_S_4838
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/153_S_4838/MPRAGE/2012-07-18_08_00_55.0/S157969 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/153_S_4838 2012-08-15
073_S_4825
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/073_S_4825/MPRAGE/2012-06-26_17_01_22.0/S155617 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/073_S_4825 2012-08-28
032_S_4823
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/032_S_4823/MPRAGE/2012-07-05_09_11_51.0/S156853 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/032_S_4823 2012-07-16
130_S_4817
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/130_S_4817/MPRAGE_SENSE2/2012-07-03_07_44_50.0/S156605 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/130_S_4817 2012-07-17
137_S_4816
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/137_S_4816/MPRAGE/2012-08-14_13_27_19.0/S160722 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/137_S_4816 2012-09-07
137_S_4815
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/137_S_4815/MPRAGE/2012-08-02_15_28_06.0/S159529 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/137_S_4815 2012-09-04
009

041_S_4629
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/041_S_4629/MPRAGE/2012-04-17_13_05_00.0/S147400 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/041_S_4629 2012-04-23
127_S_4624
137_S_4623
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/137_S_4623/MPRAGE/2012-04-04_08_17_12.0/S146005 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/137_S_4623 2012-05-04
024_S_4622
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/024_S_4622/MPRAGE/2012-03-30_10_02_56.0/S145295 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/024_S_4622 2012-04-06
153_S_4621
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/153_S_4621/MPRAGE/2012-03-28_10_22_19.0/S144994 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/153_S_4621 2012-04-17
073_S_4614
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/073_S_4614/MPRAGE/2012-03-16_08_46_55.0/S144053 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/073_S_4614 2012-04-24
072_S_4613
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/072_S_4613/MPRAGE/2012-04-02_12_59_55.0/S146058 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/072_S_4613 2012-04-12

099_S_4475
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/099_S_4475/MPRAGE/2012-02-01_11_01_33.0/S139198 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/099_S_4475 2012-03-01
002_S_4473
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_4473/MPRAGE_SENSE2/2012-02-10_08_29_21.0/S140458 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/002_S_4473 2012-02-16
007_S_4467
072_S_4465
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/072_S_4465/MPRAGE/2012-02-14_12_35_25.0/S140706 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/072_S_4465 2012-02-21
099_S_4463
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/099_S_4463/MPRAGE/2012-02-02_06_47_56.0/S139377 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/099_S_4463 2012-02-29
072_S_4462
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/072_S_4462/MPRAGE/2012-02-02_16_11_51.0/S139561 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/072_S_4462 2012-02-14
126_S_4458
141_S_4456
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/141_S_4456/MPRAGE/2012-01-17_09_40_08.0/S137389 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/141

...

/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/168_S_6821/Accelerated_Sagittal_MPRAGE_MPR_Tra/2019-10-23_15_16_31.0/S887229 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/168_S_6821 2019-12-04
301_S_6811
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/301_S_6811/Sagittal_3D_Accelerated_MPRAGE/2019-09-09_09_17_11.0/S871478 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/301_S_6811 2019-12-03
137_S_6693
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/137_S_6693/Accelerated_Sagittal_MPRAGE/2019-08-19_14_35_43.0/S862633 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/137_S_6693 2019-12-16
052_S_6832
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/052_S_6832/Accelerated_Sagittal_MPRAGE/2019-11-15_09_28_16.0/S898888 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/052_S_6832 2020-01-09
052_S_6844
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/052_S_6844/Accelerated_Sagittal_MPRAGE/2020-01-03_15_23_10.0/S909587 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/052_S_6844 2020-01-13
024_S_6846
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/024_S_6846/Accelera

021_S_4857
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/021_S_4857/Accelerated_Sag_IR-SPGR/2012-07-24_15_43_54.0/S158488 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/021_S_4857 2012-08-02
137_S_4852
012_S_4849
127_S_4844
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/127_S_4844/Accelerated_Sag_IR-SPGR/2012-07-18_08_33_01.0/S157902 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/127_S_4844 2012-08-06
128_S_4842
153_S_4838
073_S_4825
032_S_4823
130_S_4817
137_S_4816
137_S_4815
009_S_4814
018_S_4809
052_S_4807
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/052_S_4807/Accelerated_Sag_IR-SPGR/2012-07-27_13_28_25.0/S158747 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/052_S_4807 2012-08-08
123_S_4806
022_S_4805
027_S_4804
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/027_S_4804/Accelerated_SAG_IR-SPGR/2012-07-16_12_30_35.0/S157715 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/027_S_4804 2012-07-24
141_S_4803
002_S_4799
070_S_4798
023_S_4796
070_S_4793
013_S_4791
035_S_4784
067_S_4782
123_S_4780
073_S_4777
072_S_4769
067_S_47

068_S_4274
007_S_4272
041_S_4271
128_S_2130
013_S_4268
014_S_4263
109_S_4260
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/109_S_4260/Accelerated_SAG_IR-SPGR/2011-10-14_12_34_04.0/S125936 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/109_S_4260 2011-10-25
073_S_4259
035_S_4256
002_S_4251
130_S_4250
022_S_2263
021_S_4245
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/021_S_4245/Accelerated_Sag_IR-SPGR/2011-09-22_08_26_17.0/S123111 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/021_S_4245 2011-10-12
082_S_4244
023_S_4243
014_S_2308
023_S_4241
127_S_4240
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/127_S_4240/Accelerated_Sag_IR-SPGR/2011-09-23_12_04_11.0/S123238 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/127_S_4240 2011-10-05
011_S_4235
005_S_2390
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/005_S_2390/Accelerated_Sag_IR-SPGR/2011-06-27_09_49_18.0/S112697 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/005_S_2390 2011-07-08
141_S_4232
130_S_2391
002_S_4229
130_S_2403
002_S_4219
068_S_4217
073_S_4216
037_S_4214
067_S_42

018_S_2155
068_S_2171
027_S_2183
068_S_2184
073_S_2190
073_S_2191
068_S_2194
067_S_2195
067_S_2196
035_S_2199
099_S_2205
009_S_2208
127_S_2213
094_S_2216
073_S_2225
031_S_2233
024_S_2239
032_S_2240
027_S_2245
032_S_2247
068_S_2248
073_S_2264
011_S_2274
016_S_2284
067_S_2304
082_S_2307
068_S_2315
068_S_2316
013_S_2324
141_S_2333
027_S_2336
129_S_2347
126_S_2360
130_S_2373
029_S_2376
036_S_2378
022_S_2379
009_S_2381
013_S_2389
029_S_2395
057_S_2398
067_S_5160
029_S_5135
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/029_S_5135/Accelerated_SAG_IR-FSPGR/2013-04-24_10_23_19.0/S188030 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/029_S_5135 2013-05-15
128_S_5066
114_S_5047
016_S_5031
041_S_5026
082_S_5014
016_S_5007
009_S_5000
041_S_4989
012_S_4987
073_S_4986
013_S_4985
141_S_4976
041_S_4974
027_S_4966
009_S_4958
027_S_4955
031_S_4947
052_S_4945
052_S_4944
027_S_4943
072_S_4941
027_S_4936
051_S_4929
127_S_4928
027_S_4926
130_S_4925
022_S_4922
020_S_4920
027_S_4919
067_S_4918
013_S_4917
068_S_4914
057

011_S_6618
109_S_6364
109_S_6406
027_S_6640
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/027_S_6640/Accelerated_Sagittal_IR-FSPGR/2018-12-26_14_40_37.0/S770770 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/027_S_6640 2019-01-09
002_S_6652
003_S_6606
109_S_6221
035_S_6641
027_S_6643
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/027_S_6643/Accelerated_Sagittal_IR-FSPGR/2018-12-26_12_23_55.0/S770783 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/027_S_6643 2019-01-14
099_S_6632
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/099_S_6632/Accelerated_Sag_IR-FSPGR/2018-12-19_09_50_40.0/S762639 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/099_S_6632 2019-01-17
019_S_6635
109_S_6373
109_S_6376
130_S_6612
019_S_6668
168_S_6634
006_S_6651
109_S_6300
037_S_6627
341_S_6686
130_S_6688
006_S_6681
006_S_6682
131_S_6616
114_S_6597
135_S_6703
/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/135_S_6703/Accelerated_Sag_IR-FSPGR/2019-03-26_13_10_01.0/S810307 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/135_S_6703 2019-04-09
033_S_6697
002_S

## This part is for CN data

In [8]:
CN_amyloid_pos = pd.read_csv(os.path.join(csvPath,'CN_amyloid_pos.csv'))

In [9]:
CN_amyloid_pos

,PTID,EXAMDATE,ABETA_bl,AV45
0,016_S_4951,2012-11-19,1024.0,1.039290
1,070_S_4856,2012-08-03,975.8,0.937777
2,116_S_4855,2012-08-07,732.8,1.213200
3,127_S_4843,2012-08-06,NaN,1.187480
4,019_S_4835,2012-07-24,728.6,1.099920
...,...,...,...,...
104,003_S_6307,2018-08-13,NaN,1.190080
105,035_S_6551,2018-08-23,NaN,1.701010
106,168_S_6492,2018-08-30,NaN,1.296410
107,003_S_6644,2019-01-30,NaN,1.248110


In [10]:
pathRoot = "/home/ubuntu/Desktop/ADNI_MRI_Image/CM1"
niiPathRoot = '/home/ubuntu/Desktop/ADNI_MRI_Image/nii_AmyloidPos_CN'

In [11]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-SPGR' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-FSPGR' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

016_S_4951
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_4951/Accelerated_Sagittal_MPRAGE/2017-02-14_12_32_37.0/S537530 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_4951 2012-11-19
070_S_4856
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/070_S_4856/MPRAGE/2012-07-25_12_22_26.0/S158511 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/070_S_4856 2012-08-03
116_S_4855
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/116_S_4855/MPRAGE/2012-07-20_13_15_02.0/S159382 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/116_S_4855 2012-08-07
127_S_4843
019_S_4835
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/019_S_4835/MPRAGE/2012-07-10_11_09_41.0/S157253 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/019_S_4835 2012-07-24
073_S_4795
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/073_S_4795/MPRAGE_GRAPPA2_S3_DIS3D/2014-12-03_14_54_49.0/S242484 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/073_S_4795 2012-11-29
073_S_4762
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/073_S_4762/MPRAGE/2012-09-05_11_14_16.0/S166278 /home/ubuntu/Desktop/ADNI_MRI_Im

099_S_4104
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/099_S_4104/MPRAGE/2011-07-01_11_44_21.0/S113393 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/099_S_4104 2011-07-11
941_S_4100
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/941_S_4100/MPRAGE/2011-07-18_18_44_49.0/S115410 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/941_S_4100 2011-08-16
016_S_4097
014_S_4093
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/014_S_4093/MPRAGE/2011-07-07_10_13_34.0/S113844 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/014_S_4093 2011-07-25
099_S_4086
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/099_S_4086/MPRAGE/2011-06-28_13_18_33.0/S112876 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/099_S_4086 2011-07-12
003_S_4081
014_S_4080
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/014_S_4080/MPRAGE/2011-06-16_10_23_25.0/S111770 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/014_S_4080 2011-07-12
037_S_4071
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/037_S_4071/MPRAGE/2011-06-28_10_04_22.0/S112798 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/037_S_4071 2011-

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/pyth

031_S_4021
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/031_S_4021/MPRAGE/2011-04-19_13_41_24.0/S105268 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/031_S_4021 2011-05-05
114_S_6063
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/114_S_6063/Accelerated_Sagittal_MPRAGE/2017-11-13_14_10_00.0/S639225 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/114_S_6063 2018-02-20
168_S_6049
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6049/Accelerated_Sagittal_MPRAGE/2017-10-11_14_19_20.0/S619050 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6049 2018-03-28
129_S_6288
168_S_6233
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6233/AAHead_Scout_MPR_sag/2018-02-26_10_37_41.0/S662436 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6233 2018-04-18
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6233/Accelerated_Sagittal_MPRAGE/2018-02-26_10_37_41.0/S662438 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6233 2018-04-18


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER


...

032_S_6279
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/032_S_6279/Accelerated_Sagittal_MPRAGE/2018-04-04_09_37_39.0/S672893 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/032_S_6279 2018-05-29
036_S_6316
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/036_S_6316/Accelerated_Sagittal_MPRAGE/2018-04-26_08_48_36.0/S681691 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/036_S_6316 2018-05-15
177_S_6328
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/177_S_6328/Sagittal_3D_Accelerated_MPRAGE/2018-05-04_09_12_18.0/S682465 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/177_S_6328 2018-06-08
114_S_6251
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/114_S_6251/Accelerated_Sagittal_MPRAGE/2018-03-27_16_25_00.0/S672827 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/114_S_6251 2018-06-06
003_S_6260
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/003_S_6260/Accelerated_Sagittal_MPRAGE/2018-05-22_14_12_57.0/S688417 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/003_S_6260 2018-06-28
002_S_6456
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/002_S_6456/Accelerate

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER


129_S_6452
032_S_6293
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/032_S_6293/Accelerated_Sagittal_MPRAGE/2018-05-07_10_09_38.0/S682850 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/032_S_6293 2018-07-25
003_S_6307
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/003_S_6307/Accelerated_Sagittal_MPRAGE/2018-06-19_13_00_02.0/S698034 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/003_S_6307 2018-08-13
035_S_6551
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/035_S_6551/Accelerated_Sagittal_MPRAGE_Phase_A-P/2019-08-22_15_06_29.0/S866903 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/035_S_6551 2018-08-23
168_S_6492
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6492/AAHead_Scout_MPR_sag/2018-08-06_15_05_09.0/S713171 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6492 2018-08-30


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER


/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6492/Accelerated_Sagittal_MPRAGE/2018-08-06_15_05_09.0/S713177 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6492 2018-08-30
003_S_6644
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/003_S_6644/Accelerated_Sagittal_MPRAGE/2018-12-04_14_13_04.0/S755402 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/003_S_6644 2019-01-30
032_S_6699
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/032_S_6699/Accelerated_Sagittal_MPRAGE/2019-03-27_08_38_18.0/S810522 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/032_S_6699 2019-07-10


In [9]:
CN_all = pd.read_csv(os.path.join(csvPath,'CN_all.csv'))
pathRoot = "/home/ubuntu/Desktop/ADNI_MRI_Image/CM1"
niiPathRoot = '/home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN'
CN_all

,PTID,EXAMDATE
0,016_S_4951,2012-11-19
1,032_S_4921,2012-09-28
2,003_S_4900,2012-08-29
3,003_S_4872,2012-08-22
4,070_S_4856,2012-08-03
...,...,...
478,114_S_6813,2019-11-05
479,016_S_6790,2019-09-24
480,016_S_6802,2019-09-18
481,016_S_6834,2019-12-11


In [10]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    CM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(CM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-SPGR' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    CM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(CM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in CN_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-FSPGR' in serType:
                    CN_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    CM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(CM_datelist,CN_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

016_S_4951
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_4951/Accelerated_Sagittal_MPRAGE/2017-02-14_12_32_37.0/S537530 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_4951 2012-11-19
032_S_4921
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/032_S_4921/MPRAGE/2012-09-06_09_46_15.0/S166479 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/032_S_4921 2012-09-28
003_S_4900
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/003_S_4900/Accelerated_Sagittal_MPRAGE/2019-11-08_13_01_36.0/S896211 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/003_S_4900 2012-08-29
003_S_4872
070_S_4856
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/070_S_4856/MPRAGE/2012-07-25_12_22_26.0/S158511 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/070_S_4856 2012-08-03
116_S_4855
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/116_S_4855/MPRAGE/2012-07-20_13_15_02.0/S159382 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/116_S_4855 2012-08-07
127_S_4843
019_S_4835
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/019_S_4835/MPRAGE/2012-07-10_11_09_41.0/S157253 /home/ubuntu/Desk

009_S_4388
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/009_S_4388/MPRAGE/2011-12-13_15_25_59.0/S133119 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/009_S_4388 2012-01-12
007_S_4387
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/007_S_4387/Accelerated_Sagittal_MPRAGE/2017-11-01_08_55_19.0/S627305 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/007_S_4387 2011-12-19
032_S_4386
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/032_S_4386/MPRAGE/2011-12-13_10_30_18.0/S133449 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/032_S_4386 2012-01-19
029_S_4385
029_S_4384
153_S_4372
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/153_S_4372/MPRAGE/2011-11-30_16_02_08.0/S131410 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/153_S_4372 2011-12-07
129_S_4371
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/129_S_4371/MPRAGE/2012-07-17_06_28_52.0/S157773 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/129_S_4371 2011-12-06
019_S_4367
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/019_S_4367/MPRAGE/2011-11-30_10_09_24.0/S131357 /home/ubuntu/Desktop/ADNI_MRI_Image/ni

016_S_4097
014_S_4093
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/014_S_4093/MPRAGE/2011-07-07_10_13_34.0/S113844 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/014_S_4093 2011-07-25
116_S_4092
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/116_S_4092/MPRAGE/2011-06-24_08_54_40.0/S112544 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/116_S_4092 2011-07-18
082_S_4090
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/082_S_4090/MPRAGE/2011-06-20_17_57_32.0/S112177 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/082_S_4090 2011-07-11
099_S_4086
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/099_S_4086/MPRAGE/2011-06-28_13_18_33.0/S112876 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/099_S_4086 2011-07-12
035_S_4082
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/035_S_4082/MPRAGE/2011-06-28_15_17_06.0/S112905 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/035_S_4082 2011-07-05
003_S_4081
014_S_4080
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/014_S_4080/MPRAGE/2011-06-16_10_23_25.0/S111770 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/014_S_4080 2011-

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/pyth

/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6051/Accelerated_Sagittal_MPRAGE/2017-07-13_17_10_14.0/S583955 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6051 2017-08-08
067_S_6117
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/067_S_6117/Accelerated_Sagittal_MPRAGE/2017-11-21_11_00_13.0/S635418 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/067_S_6117 2017-12-13
168_S_6059
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6059/AAHead_Scout_MPR_sag/2019-11-13_15_35_46.0/S896944 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6059 2017-11-20
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/168_S_6059/AAHead_Scout_MPR_tra/2019-11-13_15_35_46.0/S896946 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/168_S_6059 2017-11-20


...

035_S_6730
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/035_S_6730/Accelerated_Sagittal_MPRAGE/2019-06-18_07_53_23.0/S833186 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/035_S_6730 2019-06-26
098_S_6734
035_S_6739
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/035_S_6739/Accelerated_Sagittal_MPRAGE/2019-06-06_07_01_43.0/S831307 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/035_S_6739 2019-07-09
116_S_6750
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/116_S_6750/Accelerated_Sagittal_MPRAGE_ND/2019-06-18_12_41_18.0/S833385 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/116_S_6750 2019-07-18
035_S_6751
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/035_S_6751/Accelerated_Sagittal_MPRAGE/2019-06-27_12_33_25.0/S838130 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/035_S_6751 2019-07-11
016_S_6773
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_6773/Accelerated_Sagittal_MPRAGE/2019-07-31_13_22_18.0/S846873 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_6773 2019-08-06
013_S_6780
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/013_S_6780

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/pyth

/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_6802/AAHead_Scout_MPR_tra/2019-08-21_16_23_33.0/S901117 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_6802 2019-09-18
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_6802/Accelerated_Sagittal_MPRAGE/2019-08-21_16_23_33.0/S901121 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_6802 2019-09-18
016_S_6834
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_6834/Accelerated_Sagittal_MPRAGE/2019-11-11_16_23_52.0/S897429 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_6834 2019-12-11
014_S_6831
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/014_S_6831/Accelerated_Sagittal_MPRAGE_ND/2019-11-07_13_42_10.0/S895594 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/014_S_6831 2020-01-06
016_S_4951
032_S_4921
003_S_4900
003_S_4872
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/003_S_4872/Sag_IR-SPGR/2012-08-02_14_42_07.0/S159407 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/003_S_4872 2012-08-22
070_S_4856
116_S_4855
127_S_4843
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/127_S_4843/

032_S_4429
023_S_4448
116_S_4453
100_S_4469
031_S_4496
109_S_4499
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/109_S_4499/Sag_IR-SPGR/2012-03-08_11_39_09.0/S143189 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/109_S_4499 2012-03-29
033_S_4505
137_S_4520
094_S_4560
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/094_S_4560/Sag_IR-SPGR/2012-03-16_13_27_19.0/S144266 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/094_S_4560 2012-04-11
014_S_4576
053_S_4578
128_S_4586
135_S_4598
016_S_4638
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/016_S_4638/Sag_IR-SPGR/2012-05-21_13_19_45.0/S151239 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/016_S_4638 2012-06-06
012_S_4643
003_S_4644
036_S_4878
016_S_4952
073_S_5023
070_S_5040
168_S_6064
129_S_6146
116_S_6133
031_S_4021
024_S_6184
037_S_6115
027_S_6183
007_S_6120
100_S_6164
036_S_6189
114_S_6057
168_S_6131
019_S_6186
006_S_6234
129_S_6228
301_S_6224
009_S_6212
006_S_6209
018_S_6207
024_S_6202
035_S_6200
014_S_6199
041_S_6192
020_S_6185
141_S_6178
009_S_6163
305_S_6157
168_S_615

301_S_6224
009_S_6212
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/009_S_6212/Accelerated_Sag_IR-FSPGR/2018-02-26_17_10_44.0/S662644 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/009_S_6212 2018-03-12
006_S_6209
018_S_6207
024_S_6202
035_S_6200
014_S_6199
041_S_6192
020_S_6185
141_S_6178
009_S_6163
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/009_S_6163/Accelerated_Sag_IR-FSPGR/2018-02-13_16_40_28.0/S658886 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/009_S_6163 2018-02-27
305_S_6157
168_S_6151
014_S_6148
014_S_6145
130_S_6105
003_S_6067
114_S_6063
037_S_6046
003_S_6014
020_S_6566
116_S_6119
003_S_6257
168_S_6049
127_S_6147
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/127_S_6147/Accelerated_Sagittal_IR-FSPGR/2018-02-13_14_19_26.0/S658857 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/127_S_6147 2018-02-20
114_S_6113
127_S_6173
/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/127_S_6173/Accelerated_Sagittal_IR-FSPGR/2018-02-05_14_10_02.0/S657038 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/127_S_6173 2018-03-07
305_S_618

## This is for AD MRI data

In [8]:
AD_all = pd.read_csv(os.path.join(csvPath,'AD_all.csv'))
pathRoot = "/home/ubuntu/Desktop/ADNI_MRI_Image/AM1"
niiPathRoot = '/home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD'
AD_all

In [13]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in AD_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType:
                    AD_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,AD_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in AD_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-SPGR' in serType:
                    AD_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,AD_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
for index,row in AD_all.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 8:
                serType = path.split('/')[7]
#                 date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'IR-FSPGR' in serType:
                    AD_date = datetime.strptime(row['EXAMDATE'],'%Y-%m-%d')
                    MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))
                    chooseDate = nearest(MM_datelist,AD_date).strftime('%Y-%m-%d_%H_%M_%S') + '.0'
                    path_date = os.path.join(path, chooseDate)
                    path_date_S = os.path.join(path_date,os.listdir(path_date)[0])
#                     print('if in',serType,date,ptid)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if os.path.exists(aimPath) == False:
                        os.mkdir(aimPath)
                    if len(os.listdir(aimPath)) == 0:
                        print(path_date_S,aimPath,row['EXAMDATE'])
                        dicom2nifti.convert_directory(path_date_S, aimPath)

135_S_5275
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/135_S_5275/MPRAGE/2013-08-05_11_21_42.0/S197268 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/135_S_5275 2013-08-27
009_S_5252
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/009_S_5252/MPRAGE/2013-07-18_09_32_48.0/S195586 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/009_S_5252 2013-08-14
016_S_5251
023_S_5241
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/023_S_5241/MPRAGE/2013-08-09_11_18_14.0/S197524 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/023_S_5241 2013-10-25
018_S_5240
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/018_S_5240/MPRAGE/2013-07-23_13_12_29.0/S196225 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/018_S_5240 2013-08-26
130_S_5231
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/130_S_5231/MPRAGE/2013-07-16_07_45_17.0/S195532 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/130_S_5231 2013-07-29
009_S_5224
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/009_S_5224/MPRAGE/2013-07-11_12_55_05.0/S195460 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/009_S_5224 2013-07-25
036_S

036_S_4894
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/036_S_4894/MPRAGE/2012-08-09_09_29_37.0/S160488 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/036_S_4894 2012-09-07
003_S_4892
016_S_4887
037_S_4879
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/037_S_4879/MPRAGE/2012-09-06_08_18_03.0/S167309 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/037_S_4879 2012-09-17
006_S_4867
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/006_S_4867/MPRAGE/2012-08-07_10_36_45.0/S159836 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/006_S_4867 2012-08-21
135_S_4863
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/135_S_4863/MPRAGE/2012-08-01_11_16_38.0/S159175 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/135_S_4863 2012-08-20
068_S_4859
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/068_S_4859/MPRAGE/2012-08-03_12_53_57.0/S159890 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/068_S_4859 2012-08-30
073_S_4853
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/073_S_4853/MPRAGE/2012-07-11_16_46_23.0/S157321 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/073_S_4853 2012-

016_S_4009
037_S_4001
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/037_S_4001/MPRAGE/2011-02-21_13_51_06.0/S100198 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/037_S_4001 2011-03-02
022_S_6013
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/022_S_6013/Accelerated_Sagittal_MPRAGE_ND/2017-04-26_13_37_50.0/S558882 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/022_S_6013 2017-05-10
168_S_6142
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/168_S_6142/AAHead_Scout_MPR_sag/2017-12-18_11_32_15.0/S645031 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/168_S_6142 2018-01-02
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/168_S_6142/Accelerated_Sagittal_MPRAGE/2017-12-18_11_32_15.0/S645034 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/168_S_6142 2018-01-02


Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER


130_S_6072
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/130_S_6072/Sagittal_3D_Accelerated_MPRAGE/2017-08-25_14_34_51.0/S601331 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/130_S_6072 2017-09-01
116_S_6100
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/116_S_6100/Accelerated_Sagittal_MPRAGE_ND/2017-10-31_11_07_13.0/S763120 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/116_S_6100 2018-01-02
114_S_6039
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/114_S_6039/Accelerated_Sagittal_MPRAGE_ND/2017-07-21_12_13_00.0/S587866 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/114_S_6039 2017-08-09
135_S_4676
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/135_S_4676/MPRAGE/2012-04-19_10_25_55.0/S147672 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/135_S_4676 2012-05-07
032_S_4755
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/032_S_4755/MPRAGE/2012-07-02_10_34_30.0/S156287 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/032_S_4755 2012-07-09
127_S_5067
131_S_5138
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/131_S_5138/MPRAGE/2013-04-02_15_38_04.0/S18719

...

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/pyth

305_S_6810
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/305_S_6810/Sagittal_3D_Accelerated_MPRAGE/2019-11-02_10_38_41.0/S894229 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/305_S_6810 2019-11-26
003_S_6833
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/003_S_6833/Accelerated_Sagittal_MPRAGE/2019-11-07_13_27_11.0/S895543 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/003_S_6833 2019-12-11
135_S_6840
016_S_6839
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/016_S_6839/Accelerated_Sagittal_MPRAGE/2019-12-03_14_08_15.0/S902460 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/016_S_6839 2020-01-13
168_S_6843
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/168_S_6843/AAHead_Scout_MPR_sag/2020-01-14_13_15_05.0/S911948 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/168_S_6843 2020-01-28
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/168_S_6843/AAHead_Scout_MPR_tra/2020-01-14_13_15_05.0/S911949 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/168_S_6843 2020-01-28
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/168_S_6843/Accelerated_Sagittal_MPRAGE/2020-01

Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dicom.py", line 120, in dicom_array_to_nifti
    results = convert_siemens.dicom_to_nifti(dicom_list, output_file)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_siemens.py", line 61, in dicom_to_nifti
    raise ConversionValidationError('TOO_FEW_SLICES/LOCALIZER')
dicom2nifti.exceptions.ConversionValidationError: TOO_FEW_SLICES/LOCALIZER
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/neuro/lib/python3.6/site-packages/dicom2nifti/convert_dir.py", line 87, in convert_directory
    convert_dicom.dicom_array_to_nifti(dicom_input, nifti_file, reorient)
  File "/home/ubuntu/anaconda3/envs/neuro/lib/pyth

135_S_5275
009_S_5252
016_S_5251
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/016_S_5251/Sag_IR-SPGR/2013-08-09_12_19_43.0/S197583 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/016_S_5251 2013-10-08
023_S_5241
018_S_5240
130_S_5231
009_S_5224
036_S_5210
068_S_5206
067_S_5205
007_S_5196
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/007_S_5196/Sag_IR-SPGR/2013-06-03_12_41_16.0/S192133 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/007_S_5196 2013-06-13
003_S_5187
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/003_S_5187/Sag_IR-SPGR/2013-06-07_13_31_22.0/S191904 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/003_S_5187 2013-06-21
082_S_5184
003_S_5165
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/003_S_5165/Sag_IR-SPGR/2013-05-16_16_54_52.0/S190110 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/003_S_5165 2013-06-12
037_S_5162
068_S_5146
128_S_5123
023_S_5120
005_S_5119
036_S_5112
100_S_5106
127_S_5095
073_S_5090
033_S_5087
013_S_5071
053_S_5070
036_S_5063
052_S_5062
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/052_S_5062/Sag_IR-SP

014_S_4039
031_S_4024
016_S_4009
/home/ubuntu/Desktop/ADNI_MRI_Image/AM1/016_S_4009/Sag_IR-SPGR/2011-06-18_11_55_32.0/S112083 /home/ubuntu/Desktop/ADNI_MRI_Image/nii_AD/016_S_4009 2011-06-28
037_S_4001
022_S_6013
168_S_6142
130_S_6072
116_S_6100
114_S_6039
135_S_4676
032_S_4755
127_S_5067
131_S_5138
053_S_5208
036_S_6179
036_S_6231
011_S_6303
135_S_6284
037_S_6216
135_S_6389
114_S_6347
037_S_6377
116_S_6543
127_S_6433
019_S_6573
135_S_6545
003_S_6264
127_S_6549
032_S_6602
032_S_6600
301_S_6592
098_S_6601
114_S_6595
136_S_4993
027_S_6648
023_S_6661
098_S_6655
126_S_6683
098_S_6658
035_S_6660
135_S_6687
019_S_6712
033_S_6705
100_S_6713
006_S_6689
126_S_6721
082_S_6690
016_S_6708
027_S_6733
168_S_6735
057_S_6746
168_S_6754
129_S_6763
013_S_6768
129_S_6784
022_S_6796
341_S_6820
168_S_6828
305_S_6810
003_S_6833
135_S_6840
016_S_6839
168_S_6843
135_S_5275
009_S_5252
016_S_5251
023_S_5241
018_S_5240
130_S_5231
009_S_5224
036_S_5210
068_S_5206
067_S_5205
007_S_5196
003_S_5187
082_S_5184
003_S_

## This part is for MCI_TAU PET data

## The code below is draft

In [14]:

dicom2nifti.convert_directory('/home/ubuntu/Desktop/ADNI_MRI_Image/CM1/041_S_6401/Accelerated_Sagittal_MPRAGE/2019-05-14_12_38_07.0/S834087','/home/ubuntu/Desktop/ADNI_MRI_Image/nii_CN/041_S_6401')


In [4]:

dicom2nifti.convert_directory('/home/ubuntu/Desktop/ADNI_MRI_Image/MTAU_PRE1/002_S_1155/AV1451_Coreg,_Avg,_Std_Img_and_Vox_Siz,_Uniform_Resolution/2017-04-24_16_34_21.0/I856322','/home/ubuntu/Desktop/ADNI_MRI_Image/nii_test/PET_002_S_1155/5_python_nii')


In [11]:
# dir1 = "002_S_0729"
# for root, dirs, files in os.walk(os.path.join(pathRoot,dir1)):
#     for name in files:
#         print(os.path.join(root, name))
#     for name in dirs:
#         print(os.path.join(root, name))

In [50]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
def stringToDate(s):
    return datetime.strptime(s.split('.')[0],'%Y-%m-%d_%H_%M_%S')
MM_datelist = list(map(lambda x: stringToDate(x), os.listdir(path)))

In [14]:
len('/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_0729/MPRAGE_SENSE2/2013-08-16_13_14_44.0/S198148'.split('/'))

10

In [ ]:
for index,row in MCI_amyloid_pos.iterrows():
    print(row['PTID'])
    for root, dirs, files in os.walk(os.path.join(pathRoot,row['PTID'])):
        for name in dirs:
            path = os.path.join(root, name)
            if len(path.split('/')) == 10:
                serType = path.split('/')[7]
                date = path.split('/')[8]
                ptid = path.split('/')[6]
#                 print(serType,date,ptid)
                if 'MP' in serType or 'FSPGR' in serType:
#                     print('if in',serType,date,ptid)
                    yearMCI = np.int(row['EXAMDATE'].split('-')[0])
                    yearMM = np.int(date.split('-')[0])
#                     print(yearMCI,yearMM)
                    aimPath = os.path.join(niiPathRoot,ptid)
                    if yearMM == yearMCI - 1 or yearMM == yearMCI + 1:
                        if os.path.exists(aimPath) == False:
                            os.mkdir(aimPath)
                        if len(os.listdir(aimPath)) == 0:
                            print(path,aimPath)
                            dicom2nifti.convert_directory(path, aimPath)

In [46]:
len('/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_0729/MPRAGE_SENSE2'.split('/'))

8

In [40]:
date_obj1 = datetime.strptime(t1,'%Y-%m-%d_%H_%M_%S')

date_obj2 = datetime.strptime(t2,'%Y-%m-%d_%H_%M_%S')

date_obj3 = datetime.strptime(t3,'%Y-%m-%d_%H_%M_%S')

In [41]:
date_obj2

datetime.datetime(2009, 8, 13, 12, 58, 57)

In [43]:
dateList = [date_obj1,date_obj2]

In [45]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))
nearest(dateList,date_obj3).strftime('%Y-%m-%d_%H_%M_%S') + '.0'

'2009-08-13_12_58_57.0'

In [34]:
t1 = '/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_0729/MPRAGE_SENSE2/2013-08-16_13_14_44.0/S198148'.split('/')[8].split('.')[0]

t2 = '/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_0729/MPRAGE_REPEAT/2009-08-13_12_58_57.0/S72366'.split('/')[8].split('.')[0]

t3 = '/home/ubuntu/Desktop/ADNI_MRI_Image/MM1/002_S_0729/MPRAGE_REPEAT/2009-08-14_12_58_57.0/S72366'.split('/')[8].split('.')[0]

In [ ]:
os.listdir?

In [ ]:
Sag_IR-FSPGR MP

In [ ]:
os.path.exists('/home/ubuntu/Desktop/ADNI_MRI_Image/nii_MCI/023_S_6702')

In [ ]:
np.int('2014-01-15_14_51_40.0'.split('-')[0])

In [ ]:
os.mkdir(os.path.join(niiPathRoot,'128_S_2130'))

In [ ]:
os.mkdir?